In [1]:
import cv2  # Import the OpenCV library for image processing
import imutils, math  # Auxiliary libraries for image processing and mathematical operations
from picamera2 import Picamera2  # Library for accessing the Raspberry Pi Camera
from IPython.display import display, Image  # Library for displaying images in Jupyter Notebook
import ipywidgets as widgets  # Library for creating interactive widgets such as buttons
import threading  # Library for creating new threads to execute tasks asynchronously
import mediapipe as mp  # Import the MediaPipe library for hand keypoint detection
from time import sleep
from logger_configurator import setup_ugv_logger

logger = setup_ugv_logger()

###################################################
###################TTS#############################
###################################################
import pyttsx3  # Importing the pyttsx3 library for text-to-speech functionality
import threading  # Importing the threading module for creating threads

# Initializing the pyttsx3 engine
engine = pyttsx3.init()

# Creating an event object to control the synchronization of audio playback
play_audio_event = threading.Event()

# Setting the speed of voice playback
engine.setProperty('rate', 180)

# Defining a function to play voice for the given text
def play_speech(input_text):
    engine.say(input_text)  # Inputting the text into the engine
    engine.runAndWait()  # Waiting for the voice output to complete
    play_audio_event.clear()  # Clearing the event to indicate voice playback is complete

def play_speech_thread(input_text):
    if play_audio_event.is_set():  # If a voice is already being played, return immediately to avoid overlapping playback
        return
    play_audio_event.set()  # Setting the event to indicate a new voice playback task has started
    # Creating a new thread to play voice using the play_speech function
    speech_thread = threading.Thread(target=play_speech, args=(input_text,))
    speech_thread.start()  # Starting the new thread to begin voice playback

###################################################
###################TTS#############################
###################################################

# Create a "Stop" button that allows the user to stop the video stream by clicking on it
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger',  # Button style: 'success', 'info', 'warning', 'danger', or ''
    tooltip='Description',
    icon='square'  # FontAwesome icon name (without the `fa-` prefix)
)

# Initialize MediaPipe drawing utilities and hand keypoint detection model
mpDraw = mp.solutions.drawing_utils
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1)  # Initialize the hand keypoint detection model to detect up to one hand


def detect_mano_cornuta(fp):
    if fp["wrist_y"] > fp["index_tip_y"] and fp["wrist_y"] > fp["pinky_tip_y"]: # handfläche unterhalb der finger
        if fp["index_tip_y"] < fp["index_dip_y"] < fp["index_pip_y"] < fp["index_mcp_y"]: # zeigefinger gestreckt
            if fp["pinky_tip_y"] < fp["pinky_dip_y"] < fp["pinky_pip_y"] < fp["pinky_mcp_y"]: # kleiner finger gestreckt
                if fp["middle_tip_y"] > fp["index_mcp_y"] and fp["ring_tip_y"] > fp["ring_mcp_y"]: # Mittel- Ringerfingerspitze unterhalb von Handballen
                    logger.info("mano cornuta")
                    play_speech("mano cornuta")
                    sleep(1)

def detect_mano_cornuta_neg(fp):
    if fp["wrist_y"] < fp["index_tip_y"] and fp["wrist_y"] < fp["pinky_tip_y"]: # handfläche unterhalb der finger
        if fp["index_tip_y"] > fp["index_dip_y"] > fp["index_pip_y"] > fp["index_mcp_y"]: # zeigefinger gestreckt
            if fp["pinky_tip_y"] > fp["pinky_dip_y"] > fp["pinky_pip_y"] > fp["pinky_mcp_y"]: # kleiner finger gestreckt
                if fp["middle_tip_y"] < fp["index_mcp_y"] and fp["ring_tip_y"] < fp["ring_mcp_y"]: # Mittel- Ringerfingerspitze unterhalb von Handballen
                    logger.info("mano cornuta")
                    play_speech("mano cornuta")
                    sleep(1)


def showPos(x, y, z):
    logger.info(f'x: ${x} - x-x-x')
    logger.info("---")
    logger.info(f'y: ${y} --- y-y-y')
    logger.info("---")
    logger.info(f'z: ${z} ----- z-z-z')
    logger.info("---")
    #detect_mano_carnuta(fp)
    sleep(0.5)
    

# Define the display function to process video frames and perform hand keypoint detection
def view(button):
    # If you are using a CSI camera, uncomment the picam2 related code below, 
    # and comment out the camera related code.
    # This is because the latest version of OpenCV (4.9.0.80) no longer supports CSI cameras, 
    # and you need to use picamera2 to capture camera images.
    
    picam2 = Picamera2()  # Create an instance of Picamera2
    picam2.configure(picam2.create_video_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))  # Configure camera parameters
    picam2.start()  # Start the camera
    
    # camera = cv2.VideoCapture(-1) 
    # camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    # camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    display_handle=display(None, display_id=True)  # Create a display handle to update the displayed image
    
    while True:
        frame = picam2.capture_array()
        # _, frame = camera.read()
        # frame = cv2.flip(frame, 1) # If your camera reverses your image

        img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        results = hands.process(img)

        # If hand keypoints are detected
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:  # Iterate through each detected hand
                # Draw hand keypoints
                for id, lm in enumerate(handLms.landmark):
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)  # Calculate the position of the keypoint in the image
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), -1)  # Draw a circle at the keypoint position

                
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)  # Draw hand skeleton connections
                # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

                target_pos = handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]

                fp = { # finger position
                    # Wrist
                    "wrist_x": handLms.landmark[mpHands.HandLandmark.WRIST].x,
                    "wrist_y": handLms.landmark[mpHands.HandLandmark.WRIST].y,
                    "wrist_z": handLms.landmark[mpHands.HandLandmark.WRIST].z,
                    
                    # Thumb
                    "thumb_cmc_x": handLms.landmark[mpHands.HandLandmark.THUMB_CMC].x,
                    "thumb_cmc_y": handLms.landmark[mpHands.HandLandmark.THUMB_CMC].y,
                    "thumb_cmc_z": handLms.landmark[mpHands.HandLandmark.THUMB_CMC].z,
                
                    "thumb_mcp_x": handLms.landmark[mpHands.HandLandmark.THUMB_MCP].x,
                    "thumb_mcp_y": handLms.landmark[mpHands.HandLandmark.THUMB_MCP].y,
                    "thumb_mcp_z": handLms.landmark[mpHands.HandLandmark.THUMB_MCP].z,
                
                    "thumb_ip_x": handLms.landmark[mpHands.HandLandmark.THUMB_IP].x,
                    "thumb_ip_y": handLms.landmark[mpHands.HandLandmark.THUMB_IP].y,
                    "thumb_ip_z": handLms.landmark[mpHands.HandLandmark.THUMB_IP].z,
                
                    "thumb_tip_x": handLms.landmark[mpHands.HandLandmark.THUMB_TIP].x,
                    "thumb_tip_y": handLms.landmark[mpHands.HandLandmark.THUMB_TIP].y,
                    "thumb_tip_z": handLms.landmark[mpHands.HandLandmark.THUMB_TIP].z,
                
                    # Index finger
                    "index_mcp_x": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_MCP].x,
                    "index_mcp_y": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_MCP].y,
                    "index_mcp_z": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_MCP].z,
                
                    "index_pip_x": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_PIP].x,
                    "index_pip_y": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_PIP].y,
                    "index_pip_z": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_PIP].z,
                
                    "index_dip_x": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_DIP].x,
                    "index_dip_y": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_DIP].y,
                    "index_dip_z": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_DIP].z,
                
                    "index_tip_x": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x,
                    "index_tip_y": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y,
                    "index_tip_z": handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].z,
                
                    # Middle finger
                    "middle_mcp_x": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].x,
                    "middle_mcp_y": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].y,
                    "middle_mcp_z": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_MCP].z,
                
                    "middle_pip_x": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_PIP].x,
                    "middle_pip_y": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_PIP].y,
                    "middle_pip_z": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_PIP].z,
                
                    "middle_dip_x": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_DIP].x,
                    "middle_dip_y": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_DIP].y,
                    "middle_dip_z": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_DIP].z,
                
                    "middle_tip_x": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].x,
                    "middle_tip_y": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].y,
                    "middle_tip_z": handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].z,
                
                    # Ring finger
                    "ring_mcp_x": handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].x,
                    "ring_mcp_y": handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].y,
                    "ring_mcp_z": handLms.landmark[mpHands.HandLandmark.RING_FINGER_MCP].z,
                
                    "ring_pip_x": handLms.landmark[mpHands.HandLandmark.RING_FINGER_PIP].x,
                    "ring_pip_y": handLms.landmark[mpHands.HandLandmark.RING_FINGER_PIP].y,
                    "ring_pip_z": handLms.landmark[mpHands.HandLandmark.RING_FINGER_PIP].z,
                
                    "ring_dip_x": handLms.landmark[mpHands.HandLandmark.RING_FINGER_DIP].x,
                    "ring_dip_y": handLms.landmark[mpHands.HandLandmark.RING_FINGER_DIP].y,
                    "ring_dip_z": handLms.landmark[mpHands.HandLandmark.RING_FINGER_DIP].z,
                
                    "ring_tip_x": handLms.landmark[mpHands.HandLandmark.RING_FINGER_TIP].x,
                    "ring_tip_y": handLms.landmark[mpHands.HandLandmark.RING_FINGER_TIP].y,
                    "ring_tip_z": handLms.landmark[mpHands.HandLandmark.RING_FINGER_TIP].z,
                
                    # Pinky
                    "pinky_mcp_x": handLms.landmark[mpHands.HandLandmark.PINKY_MCP].x,
                    "pinky_mcp_y": handLms.landmark[mpHands.HandLandmark.PINKY_MCP].y,
                    "pinky_mcp_z": handLms.landmark[mpHands.HandLandmark.PINKY_MCP].z,
                
                    "pinky_pip_x": handLms.landmark[mpHands.HandLandmark.PINKY_PIP].x,
                    "pinky_pip_y": handLms.landmark[mpHands.HandLandmark.PINKY_PIP].y,
                    "pinky_pip_z": handLms.landmark[mpHands.HandLandmark.PINKY_PIP].z,
                
                    "pinky_dip_x": handLms.landmark[mpHands.HandLandmark.PINKY_DIP].x,
                    "pinky_dip_y": handLms.landmark[mpHands.HandLandmark.PINKY_DIP].y,
                    "pinky_dip_z": handLms.landmark[mpHands.HandLandmark.PINKY_DIP].z,
                
                    "pinky_tip_x": handLms.landmark[mpHands.HandLandmark.PINKY_TIP].x,
                    "pinky_tip_y": handLms.landmark[mpHands.HandLandmark.PINKY_TIP].y,
                    "pinky_tip_z": handLms.landmark[mpHands.HandLandmark.PINKY_TIP].z,
                }

        



        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))

        #showPos(target_pos.x, target_pos.y, target_pos.z)
        detect_mano_cornuta(fp)
        detect_mano_cornuta_neg(fp)
        
        if stopButton.value==True:
            picam2.close() # If yes, close the camera
            # cv2.release() # If yes, close the camera
            display_handle.update(None)

# Display the "Stop" button and start a thread to execute the display function
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

[0:26:18.128935873] [4565]  INFO Camera camera_manager.cpp:330 libcamera v0.5.2+99-bfd68f78
[0:26:18.178142292] [4566]  INFO IPAProxy ipa_proxy.cpp:180 Using tuning file /usr/share/libcamera/ipa/rpi/vc4/ov5647.json
[0:26:18.187121408] [4566]  INFO Camera camera_manager.cpp:220 Adding camera '/base/soc/i2c0mux/i2c@1/ov5647@36' for pipeline handler rpi/vc4
[0:26:18.187244388] [4566]  INFO RPI vc4.cpp:440 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media3 and ISP device /dev/media0
[0:26:18.187349831] [4566]  INFO RPI pipeline_base.cpp:1107 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:26:18.199153006] [4565]  INFO Camera camera.cpp:1215 configuring streams: (0) 640x480-XRGB8888/SMPTE170M/Rec709/None/Full (1) 640x480-SGBRG10_CSI2P/RAW
[0:26:18.199574686] [4566]  INFO RPI vc4.cpp:615 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 640x480-SGBRG10_1X10/RAW - Selected unicam format: 640x480-pGAA/RAW


None